In [72]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import datetime

In [73]:
url = 'http://api.openweathermap.org/data/2.5/weather?'  #lat=-34.798094&lon=-58.384517&exclude=hourly,daily&appid={API key}
api_key = '068679015dfe8935454115faabc2a3ff'

In [74]:
# Parámetros de la solicitud
params = {
    #'lat': -43.79,  # Cambia 'Ciudad,País' por la ciudad y país que desees
    #'lon': -58.38,
    'q':['Almirante Brown','Argentina'],
    'lang':'es',
    'appid': api_key,
    #'exclude':['minutely','alerts'],
    'units': 'metric'  # Cambia a 'imperial' si prefieres unidades en Fahrenheit
}

In [75]:
# Realizar la solicitud GET a la API
response = requests.get(url, params=params)
data = response.json()

In [76]:
data

{'coord': {'lon': -58.45, 'lat': -34.6667},
 'weather': [{'id': 804,
   'main': 'Clouds',
   'description': 'nubes',
   'icon': '04n'}],
 'base': 'stations',
 'main': {'temp': 25.01,
  'feels_like': 25.26,
  'temp_min': 21.82,
  'temp_max': 25.66,
  'pressure': 1017,
  'humidity': 65},
 'visibility': 10000,
 'wind': {'speed': 3.13, 'deg': 74, 'gust': 4.47},
 'clouds': {'all': 100},
 'dt': 1708737257,
 'sys': {'type': 2,
  'id': 2008409,
  'country': 'AR',
  'sunrise': 1708680919,
  'sunset': 1708727969},
 'timezone': -10800,
 'id': 3436395,
 'name': 'Almirante Brown',
 'cod': 200}

In [77]:


# Define el JSON proporcionado


# Crear DataFrame
df = pd.json_normalize(data)

# Mostrar el DataFrame
df.head(4)

,weather,base,visibility,dt,timezone,id,name,cod,coord.lon,coord.lat,...,main.humidity,wind.speed,wind.deg,wind.gust,clouds.all,sys.type,sys.id,sys.country,sys.sunrise,sys.sunset
0,"[{'id': 804, 'main': 'Clouds', 'description': ...",stations,10000,1708737257,-10800,3436395,Almirante Brown,200,-58.45,-34.6667,...,65,3.13,74,4.47,100,2,2008409,AR,1708680919,1708727969


convierto la fecha unix a formato tradicional y lo ajusto a la zona horaria de la cuidad

In [78]:
fecha_unix = int(df['dt'])
# Convertir la fecha Unix a formato legible
fecha_legible = datetime.datetime.utcfromtimestamp(fecha_unix)


/tmp/ipykernel_15609/3111975765.py:1: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  fecha_unix = int(df['dt'])


In [79]:
# Zona horaria UTC-3
utc_offset = datetime.timedelta(hours=-3)
fecha_legible = fecha_legible + utc_offset #acomodamos la zona horaria

In [80]:
print(fecha_legible)

2024-02-23 22:14:17


extraemos los datos que necesitamos


In [81]:
temperatura_ = df['main.temp']
sensacion_termica_ = df['main.feels_like']
presion_ = df['main.pressure']
fecha_ = fecha_legible

In [82]:
fecha_

datetime.datetime(2024, 2, 23, 22, 14, 17)

In [83]:
temperatura.dtype

dtype('float64')

intentamos conectar con la base de datos

In [84]:
# Definir los valores a insertar
fecha = pd.to_datetime(fecha_).strftime('%Y-%m-%d')  # Convertir a formato YYYY-MM-DD
temperatura = temperatura_[0]
sensacion_termica = sensacion_termica_[0]
presion = presion_[0]

In [85]:
fecha

'2024-02-23'

In [86]:
import mysql.connector

# Conectarse al servidor MySQL
connection = mysql.connector.connect(
    host="localhost",
    user="root",
    password="GibraN@1980",
    database="sensor"
)

# Crear el cursor
cursor = connection.cursor()

# Definir los valores a insertar
#fecha = "2024-02-23"
#temperatura = 25.5
#sensacion_termica = 26.0
#presion = 1013.0

# Crear la consulta SQL
sql = """
INSERT INTO BROWN (fecha, temperatura, sensacion_termica, presion)
VALUES (%s, %s, %s, %s)
"""

# Ejecutar la consulta con los valores
cursor.execute(sql, (fecha, float(temperatura), float(sensacion_termica), float(presion)))


# Confirmar la inserción
connection.commit()

# Cerrar cursor y conexión
cursor.close()
connection.close()

print("Registro insertado exitosamente!")


Registro insertado exitosamente!
